In [59]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from sklearn.metrics import accuracy_score
from keras import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [60]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
DIRECTORY = '/content/gdrive/MyDrive/Colab Notebooks/'
DATA = DIRECTORY + 'vehicle_data/' 

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
df = pd.read_csv(DATA + 'records_all_unix_csv.csv')
df.head()

,record_id,timezone,created,modified,occurred_from,occurred_from_unix,occurred_from_hour,occurred_to,lat,lon,...,weather,light,Severity,driverIncidentDetails_id,Main cause,Description,Collision-type,Email of Encoder,Reporting Agency,Location Approximate
0,a913cced-1b3d-41ed-b5e6-52ba87816c49,UTC,2021-10-15 08:49:04,2021-10-15 08:49:04,2016-09-15 04:55:52,1473915352,4,2021-10-15 08:41:52,8.503039,124.642709,...,clear-day,dawn,Property Damage,7e5037c0-8383-434c-a40d-5ff2600df7f9,Human error,MC vs. PUJ,Side-swipe,rta.cdo@gmail.com,Other,Yes
1,cdf2e78c-a345-4ecb-9299-49aa93eacc1c,UTC,2021-10-15 08:38:18,2021-10-15 08:50:19,2016-12-11 03:32:50,1481427170,3,2021-10-15 08:32:50,8.496025,124.625377,...,clear-day,day,Injury,37c04e5d-238b-4cd1-bbdd-2b0ea88a38a6,Human error,NaN,Side-swipe,rta.cdo@gmail.com,Other,Yes
2,04660c01-6070-4e6e-850d-f2aeed151eb7,UTC,2019-01-01 12:55:33,2019-01-01 12:55:33,2019-01-01 07:01:36,1546326096,7,2019-01-01 07:01:36,14.599614,121.049595,...,partly-cloudy-day,day,Property Damage,d271e8b3-155a-41d6-9226-02c99e97e782,Human error,NaN,Side-swipe,NaN,MMDA Metrobase,NaN
3,b1bb4e9e-981f-4101-87f7-86ac494c3719,UTC,2019-01-01 13:01:52,2019-01-01 13:01:52,2019-01-01 07:06:27,1546326387,7,2019-01-01 07:06:27,14.536532,120.988870,...,partly-cloudy-day,day,Property Damage,5a9bafeb-bf22-4d29-910d-19c349353d9b,Human error,NaN,Side-swipe,NaN,MMDA Metrobase,NaN
4,d12db962-0464-4944-b376-f993d272ee2b,UTC,2019-01-01 13:03:27,2019-01-01 13:03:27,2019-01-01 08:23:46,1546331026,8,2019-01-01 08:23:46,14.631424,121.074142,...,partly-cloudy-day,day,Property Damage,3b7fcdf1-c853-4699-95d2-49936f56875d,Human error,NaN,Side-swipe,NaN,MMDA Metrobase,NaN


In [63]:
df = df[['weather', 'light', 'Main cause', 'Collision-type', 'occurred_from_unix', 'Severity' ]]
# df = df[['lat', 'lon', 'weather', 'light', 'Main cause', 'Collision-type', 'occurred_from_unix', 'Severity' ]]
df = df.rename(columns={'Main cause': 'main_cause', 'Collision-type': 'collision_type', 'occurred_from_unix' : 'occurred_from', 'Severity' : 'severity'})
df = df.iloc[:-1]

In [64]:
df.head()

,weather,light,main_cause,collision_type,occurred_from,severity
0,clear-day,dawn,Human error,Side-swipe,1473915352,Property Damage
1,clear-day,day,Human error,Side-swipe,1481427170,Injury
2,partly-cloudy-day,day,Human error,Side-swipe,1546326096,Property Damage
3,partly-cloudy-day,day,Human error,Side-swipe,1546326387,Property Damage
4,partly-cloudy-day,day,Human error,Side-swipe,1546331026,Property Damage


In [65]:
df = df.dropna()
df.reset_index(drop=True)
df.shape

(8798, 6)

In [66]:
df

,weather,light,main_cause,collision_type,occurred_from,severity
0,clear-day,dawn,Human error,Side-swipe,1473915352,Property Damage
1,clear-day,day,Human error,Side-swipe,1481427170,Injury
2,partly-cloudy-day,day,Human error,Side-swipe,1546326096,Property Damage
3,partly-cloudy-day,day,Human error,Side-swipe,1546326387,Property Damage
4,partly-cloudy-day,day,Human error,Side-swipe,1546331026,Property Damage
...,...,...,...,...,...,...
11060,clear-day,day,Human error,Side-swipe,1644198144,Property Damage
11061,clear-day,day,Human error,Side-swipe,1644201748,Property Damage
11062,clear-day,day,Human error,Side-swipe,1644210034,Property Damage
11070,clear-day,day,Human error,Side-swipe,1644308124,Property Damage


In [67]:
df.dtypes

weather           object
light             object
main_cause        object
collision_type    object
occurred_from      int64
severity          object
dtype: object

In [69]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [70]:
X

array([['clear-day', 'dawn', 'Human error', 'Side-swipe', 1473915352],
       ['clear-day', 'day', 'Human error', 'Side-swipe', 1481427170],
       ['partly-cloudy-day', 'day', 'Human error', 'Side-swipe',
        1546326096],
       ...,
       ['clear-day', 'day', 'Human error', 'Side-swipe', 1644210034],
       ['clear-day', 'day', 'Human error', 'Side-swipe', 1644308124],
       ['clear-day', 'day', 'Human error', 'Side-swipe', 1644376737]],
      dtype=object)

In [71]:
y

array(['Property Damage', 'Injury', 'Property Damage', ...,
       'Property Damage', 'Property Damage', 'Property Damage'],
      dtype=object)

In [72]:
sorted(set(y))

['Fatal',
 'Fatal|Injury|Property Damage',
 'Fatal|Property Damage',
 'Injury',
 'Injury|Property Damage',
 'Property Damage']

In [73]:
df = pd.DataFrame(X, columns = ['weather', 'light', 'main_cause', 'collision_type', 'occurred_from'])
df

,weather,light,main_cause,collision_type,occurred_from
0,clear-day,dawn,Human error,Side-swipe,1473915352
1,clear-day,day,Human error,Side-swipe,1481427170
2,partly-cloudy-day,day,Human error,Side-swipe,1546326096
3,partly-cloudy-day,day,Human error,Side-swipe,1546326387
4,partly-cloudy-day,day,Human error,Side-swipe,1546331026
...,...,...,...,...,...
8793,clear-day,day,Human error,Side-swipe,1644198144
8794,clear-day,day,Human error,Side-swipe,1644201748
8795,clear-day,day,Human error,Side-swipe,1644210034
8796,clear-day,day,Human error,Side-swipe,1644308124


In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['weather'])], remainder='passthrough')
X = np.array(ct.fit_transform(df))


In [76]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [10])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [14])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [18])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [77]:
X[0]

array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0,
       0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 1473915352], dtype=object)

In [78]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[5 3 5 ... 5 5 5]


In [79]:
print(set(y))

{0, 1, 2, 3, 4, 5}


In [82]:
total = X.shape[0]
total

8798

In [83]:
validation_len = (int) (total * 0.10)
validation_len

879

In [84]:
percent_len = total-validation_len
percent_len

7919

In [85]:
X_validation = X[percent_len:]
X_train_test = X[:percent_len]
y_validation = y[percent_len:]
y_train_test = y[:percent_len]

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size = 0.2, random_state = 0)

In [87]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [88]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.9590243 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.69517616],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.57086206],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.58841954],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.71120928],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.93282646]])

In [89]:
ann = tf.keras.models.Sequential()
ann.add(Dense(64))
ann.add(Dropout(0.5))
ann.add(Activation('relu'))

ann.add(Dense(32))
ann.add(Dropout(0.5))
ann.add(Activation('relu'))

ann.add(Dense(32))
ann.add(Dropout(0.6))
ann.add(Activation('relu'))

ann.add(tf.keras.layers.Dense(units=6, activation='softmax'))



In [90]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.01)
ann.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
# ann.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [91]:
history = ann.fit(X_train, y_train, batch_size=16, epochs = 100, verbose=1)

Epoch 1/100
396/396 [==============================] - 1s 2ms/step - loss: 0.5552 - accuracy: 0.8819
Epoch 2/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3795 - accuracy: 0.9007
Epoch 3/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3549 - accuracy: 0.9007
Epoch 4/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3523 - accuracy: 0.9007
Epoch 5/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3404 - accuracy: 0.9007
Epoch 6/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3285 - accuracy: 0.9007
Epoch 7/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3251 - accuracy: 0.9007
Epoch 8/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3279 - accuracy: 0.9007
Epoch 9/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3152 - accuracy: 0.9007
Epoch 10/100
396/396 [==============================] - 1s 2ms/step - loss: 0.3214 - accura

In [92]:
ann.evaluate(X_test,y_test, batch_size=16)

99/99 [==============================] - 0s 2ms/step - loss: 0.3043 - accuracy: 0.9040


[0.3043420910835266, 0.9040403962135315]

In [93]:
y_pred = ann.predict(X_test)
y_pred = np.amax(y_pred, axis=1)
y_pred

array([0.55248404, 0.8508018 , 0.99705505, ..., 0.99658775, 0.99647635,
       0.82039464], dtype=float32)

In [96]:
y_pred = ann.predict(X_test, batch_size=16)
y_pred1D = y_pred.argmax(1)
print ('Accuracy on test data: ' + str(accuracy_score(y_test, y_pred1D)))

Accuracy on test data: 0.9040404040404041


VALIDATION

In [97]:
X_validation_test = sc.transform(X_validation)
y_pred_validation = ann.predict(X_validation_test, batch_size=16)
y_pred_validation1D = y_pred_validation.argmax(1)
print ('Accuracy on validation data: ' + str(accuracy_score(y_validation, y_pred_validation1D)))

Accuracy on validation data: 0.8134243458475541
